In [30]:
import networkx as nx
import numpy as np

from scipy import io as sio
from torch_geometric.data import InMemoryDataset, Data
import torch
import matplotlib.pyplot as plt
from torch_geometric.utils import (get_laplacian, to_scipy_sparse_matrix,
                                   to_undirected, to_dense_adj)
from torch_geometric.utils.num_nodes import maybe_num_nodes
import math

from collections import Counter
from scipy.special import comb

from torch_geometric.nn.models import GIN
from torch_geometric.utils.convert import to_networkx, from_networkx

## Correlation with marking and Substructure Counts

In [41]:
model = GIN(2, 32, 3, 32)
loss = torch.nn.L1Loss()

graphs = []
for _ in range(100):
    graphs.append(nx.erdos_renyi_graph(20, 0.3))

In [36]:
def count_paths(adjacency_matrix, cutoff):

    # 0. instantiate Counter
    counter = Counter({i: 0 for i in range(1,cutoff+1)})

    # 0. get nx graph
    G = nx.Graph(adjacency_matrix)

    # 1. iterate over each node and count paths towards all other nodes with larger index
    n = G.number_of_nodes()
    for source in range(n):
        targets = list(range(source+1, n))
        paths = nx.all_simple_paths(G, source=source, target=targets, cutoff=cutoff)
        path_lengths = list(map(lambda x: len(x)-1, paths))
        counter.update(path_lengths)

    # 2. arrange into list and return
    return [counter[l] for l in sorted(counter)][1:]

In [43]:
num_triangles = []
num_tailed = []
num_cyc4 = []
num_star = []
num_2paths = []
num_3paths = []
num_4paths = []

for graph in graphs:
    a = np.array(nx.adjacency_matrix(graph).todense())
    A2 = a.dot(a)
    A3 = A2.dot(a)
    tri = np.trace(A3) / 6
    num_triangles.append(tri)
    tailed = ((np.diag(A3) / 2) * (a.sum(0) - 2)).sum()
    num_tailed.append(tailed)
    cyc4 = 1 / 8 * (np.trace(A3.dot(a)) + np.trace(A2) - 2 * A2.sum())
    num_cyc4.append(cyc4)
    
    deg = a.sum(0)
    star = 0
    for j in range(a.shape[0]):
        star += comb(int(deg[j]), 3)
    num_star.append(star)
    
    paths = count_paths(a, 4)
    num_2paths.append(paths[0])
    num_3paths.append(paths[1])
    num_4paths.append(paths[2])

/var/folders/vl/x5gc1ns91bqgspfwlbtsjq0c0000gp/T/ipykernel_79394/202353619.py:10: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  a = np.array(nx.adjacency_matrix(graph).todense())


In [50]:
loss = torch.nn.L1Loss()

distances_random = []
distances_max = []
distances_min = []
cent = []

for graph in graphs:
    data = from_networkx(graph)
    data.x = torch.ones(len(G)).unsqueeze(dim=1)
    v = torch.zeros(data.x.shape[0])
    x = torch.cat((data.x, v.unsqueeze(dim=1)), dim=1)
    
    v = torch.zeros(data.x.shape[0])
    G = to_networkx(data).to_undirected()
    centralities = torch.tensor(list(nx.subgraph_centrality(G).values()))
    rn = torch.argmax(centralities).item() 
    v[rn] = 1
    x_ce = torch.cat((data.x, v.unsqueeze(dim=1)), dim=1)
    
    v = torch.zeros(data.x.shape[0])
    rn = torch.argmin(centralities).item() 
    v[rn] = 1
    x_ce_min = torch.cat((data.x, v.unsqueeze(dim=1)), dim=1)
    
    v = torch.zeros(data.x.shape[0])
    rn = np.random.randint(len(v))
    v[rn] = 1
    x_random = torch.cat((data.x, v.unsqueeze(dim=1)), dim=1)

    op = model(x, data.edge_index, data.batch)
    p = op.clone()
    op_ce = model(x_ce, data.edge_index, data.batch) 
    op_ce_min = model(x_ce_min, data.edge_index, data.batch) 
    op_random = model(x_random, data.edge_index, data.batch)
    
    distances_max.append((op.sum(dim=0) - op_ce.sum(dim=0)).abs().mean().item())
    distances_min.append((op.sum(dim=0) - op_ce_min.sum(dim=0)).abs().mean().item())
    distances_random.append((op.sum(dim=0) - op_random.sum(dim=0)).abs().mean().item())

In [51]:
substructures = [num_triangles, num_tailed, num_cyc4, num_star]
name = ['num triangles', 'num tailed', 'num 4 cycles', 'num star']

for idx, substructure in enumerate(substructures):
    print(name[idx])
    print(np.corrcoef(distances_max, substructure)[0][1])
    print(np.corrcoef(distances_min, substructure)[0][1])
    print(np.corrcoef(distances_random, substructure)[0][1])

num triangles
0.0051474159788623824
-0.09413241218598847
-0.17712497003936917
num tailed
-0.017225495286448796
-0.09616413227253401
-0.18094475420579376
num 4 cycles
-0.009213250542309084
-0.03951341179394169
-0.14532803797459062
num star
0.02544785729625549
-0.03251029589563608
-0.1419209343301108


## Correlation between different centralities and Subgraph Centrality

In [3]:
SUBGRAPH_NUM_GRAPHS = 100
parameters = [[60, 5], [60, 5], [60, 5], [60, 5]]
largest_component = True

random_state = np.random.RandomState(0)
random_regular_graphs = []
for i in range(SUBGRAPH_NUM_GRAPHS):
    m, d = parameters[random_state.choice(len(parameters))]  # choose parameters uniformly to create regular graph
    G = nx.random_regular_graph(d, m, seed=random_state)

    edges = list(G.edges())
    edges_to_remove = [edges[i] for i in random_state.choice(len(edges),m,replace=False)]  # remove m edges
    for edge in edges_to_remove:
        G.remove_edge(*edge)

    if largest_component:
        largest_cc = max(nx.connected_components(G), key=len)
        G_lcc = nx.convert_node_labels_to_integers(G.subgraph(largest_cc).copy())
        random_regular_graphs.append(G_lcc)
    else:
        random_regular_graphs.append(G)

In [4]:
correlation_pagerank = []
correlation_betweeness = []
correlation_degree = []
correlation_closeness = []

for G in random_regular_graphs:
    sc = list(nx.subgraph_centrality(G).values())
    pr = list(nx.pagerank(G).values())
    dc = list(nx.degree_centrality(G).values())
    cc = list(nx.closeness_centrality(G).values())
    bc = list(nx.betweenness_centrality(G).values())
    
    correlation_pagerank.append(np.corrcoef(pr, sc)[0][1])
    correlation_betweeness.append(np.corrcoef(bc, sc)[0][1])
    correlation_degree.append(np.corrcoef(dc, sc)[0][1])
    correlation_closeness.append(np.corrcoef(cc, sc)[0][1])

In [5]:
print(np.mean(correlation_pagerank))
print(np.std(correlation_pagerank))

0.9233506618482834
0.02499154573303128


In [6]:
print(np.mean(correlation_betweeness))
print(np.std(correlation_betweeness))

0.8005609463351633
0.07468393969910885


In [7]:
print(np.mean(correlation_degree))
print(np.std(correlation_degree))

0.9704933467786431
0.012401940735072663


In [8]:
print(np.mean(correlation_closeness))
print(np.std(correlation_closeness))

0.7859225270314133
0.06719529739829969
